## Inference code for glue tasks

In [1]:
from transformers import *
from random import seed
from random import choice
import datetime
import torch
import os
import uuid

In [2]:
# prep file to store time interval
filename = 'inferenceTextClass1.csv'
path = os.getcwd()
path = path + '/logs/' + filename
if os.path.exists(path):
    id = uuid.uuid4()
    renameFile = 'logs/' + filename.split(".")[0] + str(id) + '.csv'
    os.rename(path, renameFile)
with open(path, "w") as file1:
    row = "model" + "," + "start_time" + "," + "end_time" + "\n"
    file1.writelines(row)

In [4]:
def sample_text():
    #from goodnight stories for rebel girls
    text = "Once upon a time, there was a girl named Ada who loved machines. She also loved the idea of flying. She studied birds to work out the perfect balance between wing size and body weight. She tested out materials and tried out several designs. She never managed to soar like a bird, but she created a beautiful book full of drawings called Flyology where she recorded all of her findings. One night, Ada went to a ball. There, she met a grumpy old mathematician named Charles Babbage. Ada was a brilliant mathematician herself, and the two soon became good friends. Charles invited Ada to see a machine he had invented. He called it the Difference Engine. It could automatically add and subtract numbers. No one had ever done that before. Ada was hooked. “What if we built a machine that could make more complicated calculations?” she said. Excited, Ada and Charles started working. The machine was huge and it required an enormous steam engine. Ada wanted to go further: “What if this machine could play music and show letters as well as numbers?” She was describing a computer, way before modern computers were invented! Ada wrote the first computer program in history.\
Once upon a time, there was a girl called Mary who wore whatever clothes she wanted: boots, pants, ties, shirts. At that time, girls were expected to wear tightly laced corsets and layers of petticoats under their skirts. It was hard to move or even breathe in clothes like that. But unlike all her friends’ parents, Mary’s mom and dad thought that everyone, including girls, should wear whatever they liked. Her father, a self-taught country doctor, thought that all of his children would be happier and healthier in comfortable trousers and shirts, especially in the hot, humid summers. Mary was happy about that—she much prefered boys’ clothes anyway.\
Mary, her sisters, and her brother were encouraged by their father to study. Mary wanted to be a doctor, so she attended medical school and graduated as one of the first female doctors in the United States. Mary married a fellow doctor and she wore trousers and a coat for their wedding, because she liked that outfit more than a traditional wedding dress. When the Civil War broke out, she stepped forward to serve in the Union Army. A few times, Mary was arrested for dressing in men’s clothes. But to Mary, those were just clothes—she just wore what she wanted. She saved many lives during the Civil War and was awarded a Congressional Medal of Honor once it had ended. She wore the medal her whole life, on the collar of her coat, next to her tie.\
Once upon a time, in India, there was a little girl called Mary. Mary’s family was very poor and struggled to put food on the table. Mary wanted to help her family live a better life, so she decided to become a boxer. One day, she boldly walked up to the coach in a boxing gym. “Will you train me?” she asked. “You’re too tiny,” he said. “Go away.” But when the coach finished for the day, he found her still waiting for him by the gate. “I want to do this. Put me in the ring,” she said to him. Reluctantly, he took her on and she started training hard. She started competing, and won many fights. But she hadn’t told her parents—she didn’t want to worry them. One day, her dad read about her in a newspaper. “Is this you?” he asked, worried. “Yes,” said Mary, proudly. “What if you get hurt?” her mother asked. “We don’t have the money for doctors!” “I will work hard, and save as much as I can. Don’t worry,” Mary replied. She slept in hostels, ate vegetables and rice because she could not afford meat, skipped breakfast because she only had money for lunch and dinner, and she became a champion. Her parents watched her fights on TV. Mary won medal after medal. She even won a medal at the Olympics! She made her village proud, and was able to provide for her family, just like she had dreamed when she was a child."
    sentences = text.split(".")
    return sentences

In [5]:
device = "cpu"
if torch.cuda.is_available():
    device = "cuda"
print(device)

cuda


In [6]:
def load_model(model_name):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSequenceClassification.from_pretrained(model_name)
    return tokenizer, model

In [26]:
def single_sentence(tokenizer,model,sentences,modelname):
    model.to(device)
    start_time = datetime.datetime.now()
    for i in range(50000):
        sentence_0 = choice(sentences)

        inputs_1 = tokenizer.encode_plus(sentence_0, add_special_tokens=True, return_tensors='pt').to(device)

        pred_1 = model(inputs_1['input_ids'], token_type_ids=inputs_1['token_type_ids'])[0].argmax().item()

    end_time = datetime.datetime.now()
    print(start_time, end_time)
    row = f"{modelname},{start_time},{end_time}\n"
    with open(path, "a") as file1:
        file1.writelines(row)

In [27]:
def two_sentences(tokenizer,model,sentences, modelname):
    model.to(device)

    start_time = datetime.datetime.now()
    for i in range(50000):
        sentence_0 = choice(sentences)
        sentence_1 = choice(sentences)

        inputs_1 = tokenizer.encode_plus(sentence_0, sentence_1, add_special_tokens=True, return_tensors='pt').to(device)

        pred_1 = model(inputs_1['input_ids'], token_type_ids=inputs_1['token_type_ids'])[0].argmax().item()

    end_time = datetime.datetime.now()
    print(start_time, end_time)
    row = f"{modelname},{start_time},{end_time}\n"
    with open(path, "a") as file1:
        file1.writelines(row)

In [28]:
sentences = sample_text()

In [29]:
tokenizer, model = load_model("shivangi/MRPC_64_128_output")

In [30]:
get_ipython().system = os.system

In [31]:
## start nvidia-smi to record utilisation when inferencing, and kill at the end
!sh nvidiasmi.sh MRPC &
two_sentences(tokenizer,model,sentences, 'MRPC')
!pkill nvidia-smi

2020-08-05 12:34:04.899817 2020-08-05 12:40:03.361838


0

In [ ]:
tokenizer, model = load_model("shivangi/STS-B_64_128_output")
!sh nvidiasmi.sh STS-B &
two_sentences(tokenizer,model,sentences, 'STS-B')
!pkill nvidia-smi

tokenizer, model = load_model("shivangi/CoLA_64_128_output")
!sh nvidiasmi.sh CoLA &
single_sentence(tokenizer,model,sentences, 'CoLA')
!pkill nvidia-smi

2020-08-05 12:40:08.689517 2020-08-05 12:46:06.375373
2020-08-05 12:46:11.615604 2020-08-05 12:52:02.424696


0